In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import pickle


def plot_confusion_matrix(y_true, y_pred, label,
                          cmap=plt.cm.Blues):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data

    print(cm)

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=label, yticklabels=label,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


import pandas as pd

In [67]:
c = [1, 10, 100, 1000]
gamma = [1e-3, 1e-4,.04]
features = ['Date', 'Payrolls_3mo_vs_12mo', 'Effective_Fed_Funds_12_chg', 'CPI_All_Items_3_mo_annualised',
            '10Y_Treasury_Rate_12_chg', '3M_10Y_Treasury_Spread', 'S&P_500_Index_12_chg','Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/Data/final_features.csv')[::-1]
predicted_probability=pd.DataFrame()
past_predict = pd.DataFrame()
test_prob = pd.DataFrame()
recession_data = df[features]
#df_features.join(pd.get_dummies(df['Recession']))
output_feature =['Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']

In [68]:
x_training_data=recession_data[recession_data['Date'] <= '2018-02-01']
x_testing_data = recession_data[recession_data['Date'] >= '2018-02-01']


In [73]:
df.columns

Index(['Date', 'Non-farm_Payrolls', 'Civilian_Unemployment_Rate',
       'Effective_Fed_Funds', 'CPI_All_Items', '10Y_Treasury_Rate',
       '5Y_Treasury_Rate', '3_Month_T-Bill_Rate', 'IPI', 'S&P_500_Index',
       'Non-farm_Payrolls_3_mo_annualised', 'CPI_All_Items_3_mo_annualised',
       'IPI_3_mo_annualised', 'Non-farm_Payrolls_3_month_pchg',
       'Non-farm_Payrolls_12_month_pchg',
       'Civilian_Unemployment_Rate_12_month_pchg',
       'CPI_All_Items_3_month_pchg', 'CPI_All_Items_12_month_pchg',
       'IPI_3_month_pchg', 'IPI_12_month_pchg', 'Effective_Fed_Funds_12_chg',
       '10Y_Treasury_Rate_12_chg', 'S&P_500_Index_12_chg',
       '3M_10Y_Treasury_Spread', 'Payrolls_3mo_vs_12mo', 'IPI_3mo_vs_12mo',
       'CPI_3mo_vs_12mo', 'Recession', 'Recession_in_6mo', 'Recession_in_12mo',
       'Recession_in_24mo'],
      dtype='object')

In [69]:
X_train, X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [60]:
X_train

,Date,Payrolls_3mo_vs_12mo,Effective_Fed_Funds_12_chg,CPI_All_Items_3_mo_annualised,10Y_Treasury_Rate_12_chg,3M_10Y_Treasury_Spread,S&P_500_Index_12_chg,Recession_in_6mo,Recession_in_12mo,Recession_in_24mo
639,1967-01-01,0.988519,1.117647,1.006102,0.993492,-0.14,0.932494,0,0,0
276,1997-04-01,1.004519,1.055556,1.012606,1.058372,1.73,1.224972,0,0,0
574,1972-06-01,1.010248,0.908350,1.029302,0.937117,2.20,1.085512,0,0,1
295,1995-09-01,0.997436,1.226216,1.018500,0.831099,0.92,1.263016,0,0,0
702,1961-10-01,1.022797,0.914980,1.008046,1.007712,1.62,1.285259,0,0,0
...,...,...,...,...,...,...,...,...,...,...
425,1984-11-01,0.995888,1.009636,1.034931,0.989735,2.96,0.983053,0,0,0
62,2015-02-01,0.998312,1.571429,0.972588,0.730627,1.96,1.131786,0,0,0
556,1973-12-01,1.002948,1.866792,1.100957,1.059748,-0.71,0.826345,1,1,0
638,1967-02-01,0.983534,1.086957,1.014679,0.958592,0.07,0.951326,0,0,0


In [72]:
for i in output_feature:
    svm_ac = {}
    for x, y in [(x, y) for x in c for y in gamma]:
        k = x, y
        # initalize the'model' and passing the SVM Classifier with respect to C and gamma value
        model = svm.SVC(C=x, kernel='rbf', gamma=y, probability=True,
                        tol=1e-3,
                        class_weight='balanced')
        accuracy = cross_val_score(model, X_train.iloc[:,1:7], y_train[i], cv=10, scoring='recall')
        svm_ac[k] = np.mean(accuracy)
        print("\n{:20} {:25}".format('Parameter', 'Average accuracy'))
        print('{}\t{:25}'.format(k, np.mean(accuracy)))
    print(f"\n----{i}------")
    acc_score = max(svm_ac, key=svm_ac.get)
    print(f"The best tuning parameter{acc_score}")
    svc_model = svm.SVC(C=acc_score[0], kernel='rbf', gamma=acc_score[1], probability=True,
                        tol=1e-3, random_state=123,
                        class_weight='balanced')
   
    # fitting the model with training data
    svc_model.fit(X_train.iloc[:,1:7], y_train[i])
    # predicting the model with the test data
    y_predict = svc_model.predict(X_test.iloc[:,1:7])

    # evaluating the accuracy with the ground truth and predicted value
    model_acc = accuracy_score(y_test[i], y_predict)

    print("Model Accuracy is: {}".format(model_acc))
    # classification_report((y_true, y_pred)
    predicted_probability[i+"_probability"] = svc_model.predict_proba(x_testing_data.iloc[:,1:7])[:,1]
    past_predict[i+"_probability"]=svc_model.predict_proba(X_train.iloc[:,1:7])[:,1]
    test_prob[i+"_probability"]=svc_model.predict_proba(X_test.iloc[:,1:7])[:,1]
    filename = i+'.sv'
    pickle.dump(svc_model, open(filename, 'wb'))
      


Parameter            Average accuracy         
(1, 0.001)	       0.7410714285714286

Parameter            Average accuracy         
(1, 0.0001)	                      1.0

Parameter            Average accuracy         
(1, 0.04)	       0.8214285714285714

Parameter            Average accuracy         
(10, 0.001)	       0.7821428571428571

Parameter            Average accuracy         
(10, 0.0001)	       0.7410714285714286

Parameter            Average accuracy         
(10, 0.04)	       0.8803571428571428

Parameter            Average accuracy         
(100, 0.001)	       0.8642857142857142

Parameter            Average accuracy         
(100, 0.0001)	       0.7821428571428571

Parameter            Average accuracy         
(100, 0.04)	       0.8517857142857143

Parameter            Average accuracy         
(1000, 0.001)	       0.8339285714285714

Parameter            Average accuracy         
(1000, 0.0001)	       0.8642857142857142

Parameter            Average accuracy         
(

In [62]:

predicted_probability['Date']= x_testing_data['Date'].tolist()

In [63]:
predicted_probability

,Recession_in_6mo_probability,Recession_in_12mo_probability,Date
0,0.005174,0.059476,2018-02-01
1,0.013060,0.079108,2018-03-01
2,0.014981,0.101891,2018-04-01
3,0.014124,0.112264,2018-05-01
4,0.022037,0.165858,2018-06-01
5,0.027516,0.166773,2018-07-01
6,0.025959,0.210311,2018-08-01
7,0.025222,0.221093,2018-09-01
8,0.027930,0.180913,2018-10-01
9,0.031035,0.209405,2018-11-01


In [64]:
past_predict['Date']= X_train['Date'].tolist()
test_prob['Date']=X_test['Date'].tolist()


In [65]:
past_predict.append(test_prob)

,Recession_in_6mo_probability,Recession_in_12mo_probability,Date
0,0.627469,0.276094,1967-01-01
1,0.016652,0.038489,1997-04-01
2,0.024080,0.022761,1972-06-01
3,0.037119,0.094589,1995-09-01
4,0.018909,0.044155,1961-10-01
...,...,...,...
221,0.003691,0.019929,2012-09-01
222,0.364725,0.094444,1970-10-01
223,0.047619,0.181493,1999-12-01
224,0.001812,0.006207,2004-07-01


In [66]:
past_predict.append(predicted_probability)

,Recession_in_6mo_probability,Recession_in_12mo_probability,Date
0,0.627469,0.276094,1967-01-01
1,0.016652,0.038489,1997-04-01
2,0.024080,0.022761,1972-06-01
3,0.037119,0.094589,1995-09-01
4,0.018909,0.044155,1961-10-01
...,...,...,...
22,0.223219,0.180487,2019-12-01
23,0.378919,0.172395,2020-01-01
24,0.654310,0.152846,2020-02-01
25,0.805147,0.057668,2020-03-01


In [139]:
file=['modelRecession_in_6mo.sv','modelRecession_in_12mo.sv']

for i in file:
    svmc = pickle.load(open(i, 'rb'))
    print(svmc.predict_proba(test.iloc[:,1:6])[:,1])
    
    
    



[0.48641245]
[0.40234626]


In [131]:
test=recession_data[recession_data['Date'] == '2007-02-01']

test

,Date,Payrolls_3mo_vs_12mo,Effective_Fed_Funds_12_chg,CPI_All_Items_3_mo_annualised,10Y_Treasury_Rate_12_chg,3M_10Y_Treasury_Spread,S&P_500_Index_12_chg,Recession_in_6mo,Recession_in_12mo,Recession_in_24mo
156,2007-02-01,1.001109,1.171492,1.044813,1.032823,-0.31,1.098512,0,1,1


In [43]:
X_train,X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=2000, random_state=0,class_weight='balanced')
clf.fit(X_train.iloc[:,1:7], y_train)

/Users/sebin/anaconda3/envs/p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=2000, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [55]:
y_predict = clf.predict_proba(X_test.iloc[:,1:7])

  

In [56]:
y_predict

[array([[0.9, 0.1],
        [0.9, 0.1],
        [1. , 0. ],
        [0.9, 0.1],
        [0.8, 0.2],
        [0.8, 0.2],
        [1. , 0. ],
        [0.7, 0.3],
        [0.9, 0.1],
        [1. , 0. ],
        [1. , 0. ],
        [1. , 0. ],
        [1. , 0. ],
        [0.9, 0.1],
        [0.9, 0.1],
        [0.9, 0.1],
        [0.6, 0.4],
        [0.9, 0.1],
        [1. , 0. ],
        [0.7, 0.3],
        [0.4, 0.6],
        [1. , 0. ],
        [0.5, 0.5],
        [0.6, 0.4],
        [1. , 0. ],
        [0.8, 0.2],
        [1. , 0. ],
        [1. , 0. ],
        [0.5, 0.5],
        [0.9, 0.1],
        [1. , 0. ],
        [1. , 0. ],
        [1. , 0. ],
        [0.2, 0.8],
        [1. , 0. ],
        [0.6, 0.4],
        [1. , 0. ],
        [0.8, 0.2],
        [1. , 0. ],
        [0.9, 0.1],
        [1. , 0. ],
        [1. , 0. ],
        [0.8, 0.2],
        [0.8, 0.2],
        [1. , 0. ],
        [0.8, 0.2],
        [0.8, 0.2],
        [1. , 0. ],
        [0.5, 0.5],
        [0.8, 0.2],


In [102]:
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/website/house/sydney_new_data.csv')

In [103]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [104]:
df.iloc[1:1000,:]

,SUBURB,ADDRESS,bedroom_count,Type,dwelling_type,sold_price,Method,agency_name,Date Sold,postcode,Population,Distance from CBD,Hospital,School,Super Market,latitude,longitude
1,ABBOTSFORD,"6/431 Great North Road, Abbotsford",2,u,Apartment / Unit / Flat,735000.0,Auction,PRB Real Estate,14/05/2018,2046,5377,7.4,0,1,0,-33.854317,151.129654
2,ALEXANDRIA,"64/57 Ralph Street, Alexandria",2,u,Apartment / Unit / Flat,785000.0,Private Treaty,The Agency Eastern Suburbs,14/05/2018,2015,8264,3.5,0,3,0,-33.919315,151.197900
3,ALEXANDRIA,"64/57 Ralph Street, Alexandria",2,u,Apartment / Unit / Flat,785000.0,Private Treaty,The Agency Eastern Suburbs,14/05/2018,2015,8264,3.5,0,3,0,-33.919315,151.197900
4,ASHFIELD,"208/425 Liverpool Road, Ashfield",2,u,Apartment / Unit / Flat,795000.0,Private Treaty,McGrath Leichhardt,14/05/2018,2131,23839,7.9,0,6,0,-33.887860,151.118118
5,BEACON HILL,"56 Brooker Avenue, Beacon Hill",3,h,House,1200000.0,Private Treaty,One Agency Hazlett & Poole,14/05/2018,2100,7456,13.7,0,1,0,-33.747536,151.254990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NEWTOWN,"32/20 Fitzgerald Street, Newtown",3,u,Apartment / Unit / Flat,1010000.0,Auction,Ray White Newtown,24/05/2018,2042,15025,4.1,0,6,0,-33.891884,151.180700
996,NEWTOWN,"34 Darley Street, Newtown",3,h,Terrace,1370000.0,Private Treaty,Ray White Petersham,24/05/2018,2042,15025,4.1,0,6,0,-33.906540,151.179520
997,NORTH NARRABEEN,"24 Powderworks Road, North Narrabeen",3,h,House,1400000.0,Private Treaty,Stone Real Estate Mona Vale,24/05/2018,2101,5854,20.0,0,3,0,-33.702910,151.293300
998,PADDINGTON,"69 Harris Street, Paddington",3,h,House,4375000.0,Auction,McGrath Double Bay,24/05/2018,2021,12909,2.7,0,4,0,-33.884480,151.236786


In [83]:
df.to_csv('/Users/sebin/Desktop/companyandngo/website/melboure_house.csv')

In [105]:
df.columns

Index(['SUBURB', 'ADDRESS', 'bedroom_count', 'Type', 'dwelling_type',
       'sold_price', 'Method', 'agency_name', 'Date Sold', 'postcode',
       'Population', 'Distance from CBD', 'Hospital', 'School', 'Super Market',
       'latitude', 'longitude'],
      dtype='object')

In [106]:
df.iloc[1:1000,:].to_csv('/Users/sebin/Desktop/companyandngo/website/1000_sydney.csv')